In [1]:
import sys
sys.path.insert(0, "/home/ubuntu/data/code/Modules/")
import skimage
import numpy as np
import pickle
import gc
from albumentations import Compose,HorizontalFlip, VerticalFlip, ToFloat
from sklearn.metrics import confusion_matrix
import cv2
import matplotlib.pyplot as plt
from keras import backend
from keras.models import Sequential
from keras.losses import binary_crossentropy
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, BatchNormalization, Flatten, Dropout
from keras.optimizers import SGD,Adadelta
import DataGenerator
from albumentations import Compose,HorizontalFlip, VerticalFlip, ToFloat
from sklearn.model_selection import KFold
import random
import glob
import CNN_Module as cnn_module
import models
import DataGenerator_3d
import time
import warnings

Using TensorFlow backend.


In [747]:
import pickle
#3D
val_acc, preds = pickle.load(open('/home/ubuntu/data/code/3dCNN/3d_kfold_final_ceph5.p','rb'))

#2D
#preds, val_acc, throw1, throw2 = pickle.load(open('/home/ubuntu/data/code/2dCNN/ceph_model2_lr001.p','rb'))

'''preds = result_all[0]
kfold_val_acc = result_all[1]
kfold_acc = result_all[2]
kfold_val_loss = result_all[3]
'''

#print(len(results[3]))
#train_preds = results[1]
#test_preds = results[3]
#print(train_preds)
print(len(preds))
#print(preds)

8208


In [748]:
#split preds into test train
fold_split_idx = len(preds)//5

fold1 = dict(list(preds.items())[:fold_split_idx])
fold2 = dict(list(preds.items())[fold_split_idx:2*fold_split_idx])
fold3 = dict(list(preds.items())[2*fold_split_idx:3*fold_split_idx])
fold4 = dict(list(preds.items())[3*fold_split_idx:4*fold_split_idx])
fold5 = dict(list(preds.items())[4*fold_split_idx:])

print(len(fold1))
print(len(fold2))
print(len(fold3))
print(len(fold4))
print(len(fold5))

1641
1641
1641
1641
1644


In [788]:
#change this badboi
fold = fold5
#l = list(fold.items())
#random.shuffle(l)
#fold = dict(l)

split_idx = len(fold)//5
preds_train = dict(list(fold.items())[split_idx:])
preds_test = dict(list(fold.items())[:split_idx])

print(len(preds_train))
print(len(preds_test))

1316
328


In [789]:
# need to re-format the train results
#for 3d use preds[0], for 2d remove [0], in [preds[0] for preds in list(keys1)]
keys1, values1 = preds_train.keys(), preds_train.values()
just_names1 = [preds[0] for preds in list(keys1)]
new_train_preds = dict(zip(just_names1,values1))
train_preds = new_train_preds
print(len(train_preds))

keys2, values2 = preds_test.keys(), preds_test.values()
just_names2 = [preds[0] for preds in list(keys2)]
new_test_preds = dict(zip(just_names2,values2))
test_preds = new_test_preds
print(len(test_preds))

1316
328


In [708]:
#result = [train_preds, test_preds]
#print(result)
#pickle.dump(result,open('/home/ubuntu/data/code/chaining/3dchain_preds_final_final_cipro.p','wb'))

In [22]:
'''
import pickle
results=pickle.load(open('/home/ubuntu/data/code/chaining/3dchain_preds_final_ceph2.p','rb'))
train_preds = results[0]
test_preds = results[1]
print(len(train_preds))
print(len(test_preds))
'''

"\nimport pickle\nresults=pickle.load(open('/home/ubuntu/data/code/chaining/3dchain_preds_final_ceph2.p','rb'))\ntrain_preds = results[0]\ntest_preds = results[1]\nprint(len(train_preds))\nprint(len(test_preds))\n"

In [298]:
import Get_Simple_Fluctuations_Average_Intensity as simp

In [769]:
# finding fluc data
res_flucs_csv =  simp.flucs_from_csv('/home/ubuntu/data/cephdata/resistant/')
sus_flucs_csv = simp.flucs_from_csv('/home/ubuntu/data/cephdata/susceptible/')
#combine dictionaries
all_flucs = {**res_flucs_csv, **sus_flucs_csv} 
print(len(all_flucs))

res_lengths = simp.getlengths('/home/ubuntu/data/cephdata/resistant/')
sus_lengths = simp.getlengths('/home/ubuntu/data/cephdata/susceptible/')
all_lengths = res_lengths+sus_lengths
print(len(all_lengths))

507
449


In [770]:
# identifying labels
targets = simp.gettargets('/home/ubuntu/data/cephdata/resistant/','/home/ubuntu/data/cephdata/susceptible/')
#print(targets)
print(len(targets))

449


In [771]:
# create new dictionary with paths and lengths
result_dict = {}
i=0
for path, length, _ in all_lengths:
    i+=1
    #print(i)
    #print(length)
    #print(all_flucs[path])
    #print(targets[path])
    result_dict[path]=(length,all_flucs[path],targets[path])
print(len(result_dict))

449


In [623]:
import glue_code

In [790]:
#First find the cells in the train and test partitions
print(len(test_preds))
print(len(train_preds))
test_cells = sorted(glue_code.partition_to_keys(list(test_preds.keys())))
train_cells = sorted(glue_code.partition_to_keys(list(train_preds.keys())))
print(len(test_cells))
print(len(train_cells))

328
1316
15
58


In [791]:
#go from list of cells, to list of lists of images from each cell
cell_ims_test = glue_code.split_preds_into_cells(test_preds,test_cells)
cell_ims_train = glue_code.split_preds_into_cells(train_preds,train_cells)

#get the av score for susceptible, and proportion of images classified as susceptible
cell_predictions_test = glue_code.get_cell_predictions(cell_ims_test,test_preds)
cell_predictions_train = glue_code.get_cell_predictions(cell_ims_train,train_preds)

#glue together CNN predictions and fluctuation/av/lengths/labels
test_final = glue_code.glue_flucs_preds(cell_predictions_test,result_dict)
train_final = glue_code.glue_flucs_preds(cell_predictions_train,result_dict)
print(len(test_final))
print(len(train_final))

15
58


In [792]:
test_arr =(np.array(test_final)[:,1:].astype(float))
train_arr =  (np.array(train_final)[:,1:].astype(float))

np.random.shuffle(test_arr)
np.random.shuffle(train_arr)

In [793]:
# split off the labels which we are trying to predict, from the predictors
Xtrain = train_arr[:,:-1]
Ytrain = train_arr[:,-1]

Xtest = test_arr[:,:-1]
Ytest = test_arr[:,-1]

In [794]:
#find overall accuracy of model without additional nets
res_preds=[]
sus_preds=[]
keys, values = test_preds.keys(), test_preds.values()
keys=list(keys)
values = list(values)

for i in np.arange(len(keys)):
    if "resistant" in keys[i]:
        res_preds.append((keys[i],values[i]))
        
        #print(list(keys))
        #res_preds.append(path_preds, values)
    if "susceptible" in keys[i]:
        sus_preds.append((keys[i],values[i]))
print(len(res_preds))
print(len(sus_preds))

138
190


In [795]:
res_count = 0

sus_count = 0
for data in res_preds:
    if data[1][0] > 0.5:
        res_count += 1
for data in sus_preds:
    if data[1][1] > 0.5:
        sus_count += 1
print('res acc =', res_count/len(
    res_preds))
print('sus acc =', sus_count/len(sus_preds))
print('total acc =', (sus_count+res_count)/(len(sus_preds)+len(res_preds)))

res acc = 0.4057971014492754
sus acc = 0.9052631578947369
total acc = 0.6951219512195121


In [797]:
# train new algorithm
scores = []
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier,BaggingClassifier,GradientBoostingClassifier
for i in np.arange(1,100):

    #clf = AdaBoostClassifier()
    clf = AdaBoostClassifier(n_estimators=i, random_state=0)
    clf.fit(Xtrain, Ytrain)
    scores.append(clf.score(Xtest, Ytest))
    print(clf.score(Xtest, Ytest))
print(max(scores))

0.7333333333333333
0.7333333333333333
0.6666666666666666
0.7333333333333333
0.7333333333333333
0.8
0.8
0.8
0.8
0.8
0.8
0.8
0.8
0.8
0.8
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.8
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333333333333333
0.7333

In [745]:
# train new algorithm
#clf = RandomForestClassifier()
clf = RandomForestClassifier(n_estimators=100, random_state=0)
clf.fit(Xtrain, Ytrain)
print(clf.score(Xtest, Ytest))

0.5263157894736842


In [321]:
# train new algorithm
#clf = GradientBoostingClassifier()
clf = GradientBoostingClassifier(n_estimators=10,max_depth=1)
clf.fit(Xtrain, Ytrain)
print(clf.score(Xtest, Ytest))

0.8055555555555556


In [322]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()
clf.fit(Xtrain, Ytrain)
print(clf.score(Xtest, Ytest))

0.7777777777777778


In [323]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(Xtrain, Ytrain)
print(clf.score(Xtest, Ytest))

0.8055555555555556


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [324]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(Xtrain, Ytrain)
print(clf.score(Xtest, Ytest))

0.75
